In [355]:
import numpy as np
import shap
from sklearn.inspection import permutation_importance
import pandas as pd
from utils_ import feat_imp,reduce_mem_usage,make_feat,freq_enc,fill_cat_p
from sklearn.model_selection import cross_val_score,train_test_split
import matplotlib
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score,roc_curve
import lightgbm
import xgboost
import matplotlib.pyplot as plt
import catboost
import xgboost
from sklearn.model_selection import train_test_split
from itertools import combinations
from collections import Counter
plt.style.use('tableau-colorblind10')
from sklearn.model_selection import KFold, StratifiedKFold,GroupKFold,cross_validate
from scipy.stats import rankdata
%matplotlib inline
%config inlinebackend.figure_format = 'svg'

In [356]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [357]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [358]:
def prep_bki():
    
    bki = pd.read_csv('bki.csv')
    
    CREDIT_TYPE_list=['Consumer credit', 'Credit card', 'Car loan', 'Mortgage',
           'Loan for business development', 'Microloan']    
    cat_feat=['APPLICATION_NUMBER']
    cat_feat.extend(bki.select_dtypes(include='object').columns.tolist())
    bki_gr=bki.groupby(cat_feat, as_index=False).mean()
    
    return bki_gr

In [359]:
def prep_client_profile():
    
    client_profile = pd.read_csv('client_profile.csv ')
    
    client_profile.FAMILY_STATUS.replace('Unknown','Married', inplace=True)
    client_profile.GENDER.fillna('F',inplace=True)
    client_profile['AMT_REQ_CREDIT_BUREAU']=(client_profile.AMT_REQ_CREDIT_BUREAU_HOUR+
                                             24*client_profile.AMT_REQ_CREDIT_BUREAU_HOUR+
                                             30*24*client_profile.AMT_REQ_CREDIT_BUREAU_DAY+
                                             30*24*30*client_profile.AMT_REQ_CREDIT_BUREAU_MON+
                                             30*24*30*3*client_profile.AMT_REQ_CREDIT_BUREAU_DAY+
                                             30*24*30*3*4*client_profile.AMT_REQ_CREDIT_BUREAU_DAY)

    
    return client_profile

In [360]:
def prep_payments():
    
    payments= pd.read_csv('payments.csv')
    
    payments.DAYS_ENTRY_PAYMENT.fillna(payments.DAYS_INSTALMENT, inplace=True)
    payments.AMT_PAYMENT.fillna(payments.AMT_INSTALMENT, inplace=True)
    payments=pd.get_dummies(payments)
    cat_feat=['APPLICATION_NUMBER']
    cat_feat.extend(payments.select_dtypes(include='object').columns.tolist())
    payments_gr=payments.groupby(cat_feat, as_index=False).mean()

    return payments_gr

In [361]:
def prep_applications_history():
    
    applications_history= pd.read_csv('applications_history.csv ')
   
    cat_feat=['APPLICATION_NUMBER']
    cat_feat.extend(applications_history.select_dtypes(include='object').columns.tolist())
    
    applications_history_gr=applications_history.groupby(cat_feat, as_index=False).mean()

    return applications_history_gr

In [362]:
def prep_data(x):
    bki = prep_bki()
    client_profile=prep_client_profile()
    payments=prep_payments()
    applications_history=prep_applications_history()
       
    data_list={
        'bki_gr':bki,
        'client_profile':client_profile,
           'payments_gr':payments,
           'applications_history_gr':applications_history
          }
    
    total = x.copy()
    
    for data in data_list.values():
 
        total=total.merge(data,on='APPLICATION_NUMBER',how='left')

    prep_x=total
    return prep_x  

In [363]:
train_prep0 = prep_data(train)
train_prep0=reduce_mem_usage(train_prep0)

Memory usage of dataframe is 161.98 MB
Memory usage after optimization is: 99.38 MB
Decreased by 38.7%


In [422]:
train_prep=train_prep0.copy()


train_prep.EDUCATION_LEVEL=fill_cat_p(train_prep.EDUCATION_LEVEL)
train_prep.GENDER=fill_cat_p(train_prep.GENDER)
train_prep.FAMILY_STATUS=fill_cat_p(train_prep.FAMILY_STATUS)
train_prep.CODE_REJECT_REASON=fill_cat_p(train_prep.CODE_REJECT_REASON)
train_prep.NAME_PAYMENT_TYPE=fill_cat_p(train_prep.NAME_PAYMENT_TYPE)
train_prep.NAME_CONTRACT_STATUS=fill_cat_p(train_prep.NAME_CONTRACT_STATUS)


# train_prep=train_prep.fillna(0.01)

t=train_prep.groupby('APPLICATION_NUMBER', as_index=False).PREV_APPLICATION_NUMBER_x.count()
train_prep=train_prep.merge(t, on='APPLICATION_NUMBER',how='left')    
train_prep['TOTAL_AMOUNT_CREDIT_d'] = train_prep.TOTAL_SALARY/train_prep.AMOUNT_CREDIT_x
train_prep['TOTAL_AMOUNT_CREDIT_m'] = train_prep.TOTAL_SALARY-train_prep.AMOUNT_CREDIT_x
train_prep['TOTAL_FAMILY_SIZE'] = train_prep.TOTAL_SALARY/train_prep.FAMILY_SIZE
train_prep['AMOUNT_CREDIT_FAMILY_SIZE'] = train_prep.AMOUNT_CREDIT_x/train_prep.FAMILY_SIZE 
# train_prep['TOTAL_AMOUNT_CREDIT_SIZE'] = (train_prep.TOTAL_SALARY-train_prep.AMOUNT_CREDIT_x)/train_prep.FAMILY_SIZE 
train_prep['TOTAL_AMOUNT_CREDIT_p'] = (train_prep.TOTAL_SALARY-train_prep.AMOUNT_CREDIT_x)/train_prep.TOTAL_SALARY

# train_prep['DAYS_AMOUNT_CREDIT'] = train_prep.AMOUNT_CREDIT_x/train_prep.DAYS_CREDIT
train_prep['DAY_OVERDUE_AMOUNT_CREDIT'] = train_prep.AMOUNT_CREDIT_x/train_prep.CREDIT_DAY_OVERDUE

In [423]:
# train_prep=pd.get_dummies(train_prep)
cat_feat=train_prep.drop('TARGET',axis=1).select_dtypes(include='object').columns.tolist()
train_prep=freq_enc(train_prep, cat_feat)

In [424]:
train_prep=train_prep.groupby('APPLICATION_NUMBER', as_index=False).mean()

In [425]:
y=train_prep.TARGET
x=train_prep.drop('TARGET',axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8, random_state=101)

Обучить алгоритмы LightGBM и XGBoost, получить OOF прогнозы, оценить корреляцию прогнозов на обучающей выборке. Применить модели на тестовую выборку и оценить корреляцию.

In [426]:
params_xg = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.09,
    'reg_alpha':0.6,
    'min_child_weight':30,
#     'max_delta_step':50,
    "n_estimators": 300,
    "reg_lambda": 100,
    "max_depth": 4,
    "gamma": 4,
#     "nthread": 6,
    "seed": 101
}

In [427]:
params_cb = {
    "n_estimators": 300,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 0,
    'grow_policy':'SymmetricTree',
#     'cat_features':cat_feat,
    "max_depth": 6,
    "l2_leaf_reg": 50,
    "early_stopping_rounds": 50,
    "thread_count": 10,
    'random_seed': 101,
}

In [428]:
params_lb = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
#     "min_child_weight": 0.001,
    "max_depth": 9,
    "subsample_for_bin":10000,
    "n_estimators": 300,
    "n_jobs": 6,
    "seed": 27
}

In [429]:
lgb = lightgbm.LGBMClassifier(**params_lb)
lgb.fit(
    X=x_train,
    y=y_train,
    eval_set=[(x_train, y_train), (x_test, y_test)],
    early_stopping_rounds=100,
    eval_metric="auc",
    verbose=100
)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.725501	valid_1's auc: 0.71313
[200]	training's auc: 0.744741	valid_1's auc: 0.718715
[300]	training's auc: 0.759898	valid_1's auc: 0.72135
Did not meet early stopping. Best iteration is:
[300]	training's auc: 0.759898	valid_1's auc: 0.72135


LGBMClassifier(learning_rate=0.01, max_depth=9, metric='auc', n_estimators=300,
               n_jobs=6, objective='binary', seed=27, subsample_for_bin=10000)

In [430]:
xgb = xgboost.XGBClassifier(**params_xg)
xgb.fit(
    X=x_train,
    y=y_train,
    eval_set=[(x_train, y_train), 
              (x_test, y_test)],
    early_stopping_rounds=100,
    eval_metric="auc",
    verbose=50
)

[0]	validation_0-auc:0.58714	validation_1-auc:0.59208
[50]	validation_0-auc:0.72064	validation_1-auc:0.71857
[100]	validation_0-auc:0.73522	validation_1-auc:0.72271
[150]	validation_0-auc:0.74218	validation_1-auc:0.72385
[200]	validation_0-auc:0.74218	validation_1-auc:0.72385
[235]	validation_0-auc:0.74218	validation_1-auc:0.72385


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=4, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.09, max_delta_step=0,
              max_depth=4, min_child_weight=30, missing=nan,
              monotone_constraints='()', n_estimators=300, n_jobs=8,
              num_parallel_tree=1, random_state=101, reg_alpha=0.6,
              reg_lambda=100, scale_pos_weight=1, seed=101, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [431]:
pred_train_xgb=xgb.predict_proba(x_train)[:,1]
pred_train_lgb=lgb.predict_proba(x_train)[:,1]

In [432]:
np.corrcoef(pred_train_xgb,pred_train_lgb)

array([[1.       , 0.9706696],
       [0.9706696, 1.       ]])

In [433]:
pred_test_xgb=xgb.predict_proba(x_test)[:,1]
pred_test_lgb=lgb.predict_proba(x_test)[:,1]

In [434]:
np.corrcoef(pred_test_xgb,pred_test_lgb)

array([[1.        , 0.97249409],
       [0.97249409, 1.        ]])

In [435]:
# Достаточно высокая корреляция на тесте

Усреднить прогнозы с помощью арифмитического среднего, геометрического среднего и усреднить ранги, сделать выводы о качестве отдельных моделей и о качестве комбинации.

In [436]:
pred_train_mean=0.5*(pred_train_xgb+pred_train_lgb)
pred_test_mean=0.5*(pred_test_xgb+pred_test_lgb)
pred_train_sqrt=list(map(lambda x:(x[0]*x[1])**0.5,list(zip(pred_train_xgb,pred_train_lgb))))
pred_test_sqrt=list(map(lambda x:(x[0]*x[1])**0.5,list(zip(pred_test_xgb,pred_test_lgb))))
pred_train_rank=0.5*(rankdata(pred_train_xgb)+rankdata(pred_train_lgb))
pred_test_rank=0.5*(rankdata(pred_test_xgb)+rankdata(pred_test_lgb))

In [437]:
metrcics={'xgb ':[pred_train_xgb,pred_test_xgb],
         'lgb ':[pred_train_lgb,pred_test_lgb],
         'mean': [pred_train_mean,pred_test_mean],
         'sqrt': [pred_train_sqrt,pred_test_sqrt],
         'rank': [pred_train_rank,pred_test_rank]}

for metr,means in metrcics.items():
    
    print(metr,'train: ', round(roc_auc_score(y_train, means[0]),4),\
          'test: ',round(roc_auc_score(y_test, means[1]),4),
         '+\- : ',round(roc_auc_score(y_train, means[0])-roc_auc_score(y_test, means[1]),4))



xgb  train:  0.742 test:  0.7241 +\- :  0.0179
lgb  train:  0.7599 test:  0.7214 +\- :  0.0385
mean train:  0.7525 test:  0.7234 +\- :  0.0291
sqrt train:  0.7519 test:  0.7234 +\- :  0.0285
rank train:  0.7525 test:  0.7236 +\- :  0.0289


In [441]:
# Наиболее интересные результаты у xgb и rank

Обучить CatBoost, получить OOF прогнозы и выполнить задание 1 для трех моделей.

In [438]:
cb = catboost.CatBoostClassifier(**params_cb)
cb.fit(
    X=x_train,
    y=y_train,
    eval_set=[(x_train, y_train), (x_test, y_test)],
    
    verbose=100,
    plot=False
)

0:	test: 0.6092005	test1: 0.6030648	best: 0.6030648 (0)	total: 50.5ms	remaining: 15.1s
100:	test: 0.7151446	test1: 0.7177755	best: 0.7177755 (100)	total: 4.35s	remaining: 8.57s
200:	test: 0.7280594	test1: 0.7228348	best: 0.7228348 (200)	total: 8.65s	remaining: 4.26s
299:	test: 0.7375204	test1: 0.7241544	best: 0.7242113 (298)	total: 12.9s	remaining: 0us

bestTest = 0.7242113342
bestIteration = 298

Shrink model to first 299 iterations.


In [439]:
pred_train_cb=cb.predict_proba(x_train)[:,1]
pred_test_cb=cb.predict_proba(x_test)[:,1]

In [440]:
print('xgb-cb : train ',round(np.corrcoef(pred_train_xgb,pred_train_cb)[0,1],4),
      'test ', round(np.corrcoef(pred_train_xgb,pred_train_cb)[0,1],4),
     '\nlgb-cb : train ',round(np.corrcoef(pred_train_lgb,pred_train_cb)[0,1],4),
      'test ', round(np.corrcoef(pred_train_lgb,pred_train_cb)[0,1],4),)

xgb-cb : train  0.965 test  0.965 
lgb-cb : train  0.9489 test  0.9489


In [349]:
# Неплохая корреляция с xgb

Выполнить задание 2 для трех моделей.

In [442]:
metrcics['cb']=[pred_train_cb,pred_test_cb]


In [443]:
pred_train_mean=(pred_train_xgb+pred_train_lgb+pred_train_cb)/3
pred_test_mean=(pred_test_xgb+pred_test_lgb+pred_test_cb)/3
pred_train_sqrt=list(map(lambda x:(x[0]*x[1]*x[2])**(2/3),list(zip(pred_train_xgb,pred_train_lgb,pred_train_cb))))
pred_test_sqrt=list(map(lambda x:(x[0]*x[1]*x[2])**(2/3),list(zip(pred_test_xgb,pred_test_lgb,pred_test_cb))))
pred_train_rank=(rankdata(pred_train_xgb)+rankdata(pred_train_lgb)+rankdata(pred_train_cb))/3
pred_test_rank=(rankdata(pred_test_xgb)+rankdata(pred_test_lgb)+rankdata(pred_test_cb))/3

In [444]:
metrcics={'xgb ':[pred_train_xgb,pred_test_xgb],
         'lgb ':[pred_train_lgb,pred_test_lgb],
         'cb  ':[pred_train_cb,pred_test_cb],
         'mean': [pred_train_mean,pred_test_mean],
         'sqrt': [pred_train_sqrt,pred_test_sqrt],
         'rank': [pred_train_rank,pred_test_rank]}

for metr,means in metrcics.items():
    
    print(metr,'train: ', round(roc_auc_score(y_train, means[0]),4),\
          'test: ',round(roc_auc_score(y_test, means[1]),4),
         '+\- : ',round(roc_auc_score(y_train, means[0])-roc_auc_score(y_test, means[1]),4))

xgb  train:  0.742 test:  0.7241 +\- :  0.0179
lgb  train:  0.7599 test:  0.7214 +\- :  0.0385
cb   train:  0.7375 test:  0.7242 +\- :  0.0133
mean train:  0.7493 test:  0.7248 +\- :  0.0244
sqrt train:  0.7488 test:  0.7249 +\- :  0.0239
rank train:  0.7489 test:  0.7249 +\- :  0.024


In [350]:
# У cb неплохие показатели, композиционные алгоритмы подрасли.

(опция) Объединить OOF-прогнозы для трех моделей и обучить алгоритм Логистической регрессии (и любой другой, на ваше усмотрение). Сделать выводы о достигаемом качестве, сравнить достигаемое качество с качеством отдельных моделей и моделей, полученных в п.2 и п.4.

In [342]:
rc =LogisticRegression()

In [343]:
pred_train_union=np.vstack((pred_train_xgb,pred_train_lgb,pred_train_cb)).T
pred_test_union=np.vstack((pred_test_xgb,pred_test_lgb,pred_test_cb)).T

In [344]:
rc.fit(pred_train_union, y_train)
pred_train_rc = rc.predict_proba(pred_test_union)
roc_auc_score(y_test,pred_train_rc[:,1])

0.7138907847441142

In [351]:
# такое 

In [445]:
vc=VotingClassifier(estimators=[('xgb',xgb),('lgb',lgb),('cb', cb)], voting='soft')

In [446]:
vc.fit(x_train, y_train)
roc_auc_score(y_test,vc.predict_proba(x_test)[:,1])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


0.724766422956113

In [447]:
# Неплохо

(опция) Обучить алгоритмRandomForest (желательно подтюнить параметры) и добавить к построенным ранее моделям. Выполнить задание 5.

In [332]:
rf =RandomForestClassifier(n_estimators=300,  max_depth=7,random_state=101 )
rf.fit(x_train, y_train)



RandomForestClassifier(max_depth=7, n_estimators=300, random_state=101)

In [325]:
pred_test_rf = rf.predict_proba(x_test)
pred_train_rf = rf.predict_proba(x_train)

roc_auc_score(y_train,pred_train_rf[:,1]), roc_auc_score(y_test,pred_test_rf[:,1])

(0.7373674511753207, 0.7147431231371764)

In [334]:
vc=VotingClassifier(estimators=[('xgb',xgb),('lgb',lgb),('cb', cb),('rf',rf)], voting='soft')

In [335]:
vc.fit(x_train, y_train)
roc_auc_score(y_test,vc.predict_proba(x_test)[:,1])

0.7239962762160652

In [448]:
# Пойдет